In [48]:
import numpy as np
import matplotlib.pyplot as plt
import time
from mpl_toolkits import mplot3d
#%matplotlib notebook
from matplotlib.animation import FuncAnimation
from IPython import display

E = 1e4
A = 0.111

nodes = []
bars =[]

### Nodes

cas = 3 # 2 ou 3

if cas == 3 : 
    h = 10
    H = 40

    nodes.append([0,0,0])
    nodes.append([1,0,0])
    nodes.append([1,1,0])
    nodes.append([0,1,0])

    nodes.append([0,0,h])
    nodes.append([1,0,h])
    nodes.append([1,1,h])
    nodes.append([0,1,h])

    nodes.append([0.5,0.5,H])

    ### Bars
    #socle 1
    bars.append([0,1])
    bars.append([1,2])
    bars.append([4,3])
    bars.append([3,0])
    #socle 2
    bars.append([4,5])
    bars.append([5,6])
    bars.append([6,7])
    bars.append([7,4])
    #connection 1
    bars.append([0,4])
    bars.append([1,5])
    bars.append([2,6])
    bars.append([3,7])
    #connection 2
    bars.append([4,8])
    bars.append([5,8])
    bars.append([6,8])
    bars.append([7,8])
    #renforcement
    bars.append([0,5])
    bars.append([1,6])
    bars.append([2,7])
    bars.append([3,4])
    
    nodes = np.array(nodes).astype(float)
    bars = np.array(bars)
    
    #Apply force
    P = np.zeros_like(nodes)
    P[8,2] = 10

    # support displacement
    Ur = [0,0,0,0,0,0,0,0,0,0,0,0]

    DOFCON = np.ones_like(nodes).astype(int)
    DOFCON[1,:] = 0
    DOFCON[2,:] = 0
    DOFCON[3,:] = 0
    DOFCON[4,:] = 0
    
elif cas == 2 :
    nodes.append([0,120])
    nodes.append([120,120])
    nodes.append([240,120])
    nodes.append([360,120])
    nodes.append([0,0])
    nodes.append([120,0])
    nodes.append([240,0])
    nodes.append([360,0])
    
    bars.append([0,1])
    bars.append([1,2])
    bars.append([2,3])
    bars.append([4,5])
    bars.append([5,6])
    bars.append([6,7])
    
    bars.append([5,1])
    bars.append([6,2])
    bars.append([7,3])
    
    bars.append([0,5])
    bars.append([4,1])
    bars.append([1,6])
    bars.append([5,2])
    bars.append([2,7])
    bars.append([6,3])
    
    nodes = np.array(nodes).astype(float)
    bars = np.array(bars)
    
    #Apply force
    P = np.zeros_like(nodes)
    P[7,1] = -10

    # support displacement
    Ur = [0,0,0,0]
    
    DOFCON = np.ones_like(nodes).astype(int)
    DOFCON[0,:] = 0
    DOFCON[4,:] = 0

print(len(bars))

#plt.ion()
def plot3D(nodes,c,lt,lw,lg) : 
    fig = plt.figure(figsize=(8,6))
    #plt.gca(projection='3d')
    ax = fig.add_subplot(111, projection='3d')
    for i in range(len(bars)) :
        xi,xj = nodes[bars[i,0]-1,0],nodes[bars[i,1]-1,0]
        yi,yj = nodes[bars[i,0]-1,1],nodes[bars[i,1]-1,1]
        zi,zj = nodes[bars[i,0]-1,2],nodes[bars[i,1]-1,2]
        line, = ax.plot([xi,xj],[yi,yj],[zi,zj],color = c, lw = lw, linestyle = lt)
        line.set_label(lg)
        #plt.draw()
        #fig.canvas.draw()
        #plt.pause(0.1)
        #ax.cla()
        #fig.canvas.flush_events()
    ax.set_title("Structure")
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_box_aspect([1,1,1])
    geom3D(nodes,ax)
    
def plot2D(nodes,c,lt,lw,lg) : 
    fig = plt.figure(figsize=(8,6))
    ax = fig.add_subplot(111)
    for i in range(len(bars)) :
        xi,xj = nodes[bars[i,0],0],nodes[bars[i,1],0]
        yi,yj = nodes[bars[i,0],1],nodes[bars[i,1],1]
        line, = ax.plot([xi,xj],[yi,yj],color = c, lw = lw, linestyle = lt)
        line.set_label(lg)
        #plt.draw()
        #fig.canvas.draw()
        #plt.pause(0.1)
        #ax.cla()
        #fig.canvas.flush_events()
    ax.set_title("Atom velocity distribution")
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    geom2D(nodes,ax)

def geom3D(NL,ax) : 
    x = [x for x in NL[:,0]]
    y = [y for y in NL[:,1]]
    z = [z for z in NL[:,2]]
    size = 400
    offset = size/4000.
    ax.scatter(x, y, z, c='y', s=size, zorder=1)
    for i, location in enumerate(zip(x,y)):
        #plt.annotate(i+1, (location[0]-offset, location[1]-offset), zorder=10)
        ax.text(x[i],y[i],z[i],str(i+1),size=20,zorder=2,color = "k")

def geom2D(NL,ax) : 
    x = [x for x in NL[:,0]]
    y = [y for y in NL[:,1]]
    size = 400
    offset = size/4000.
    ax.scatter(x, y, c='y', s=size, zorder=1)
    for i, location in enumerate(zip(x,y)):
        #plt.annotate(i+1, (location[0]-offset, location[1]-offset), zorder=10)
        ax.text(x[i],y[i],str(i+1),size=20,zorder=2,color = "k")

def truss_analyse2D():
    NN = len(nodes)
    NE = len(bars)
    DOF = 2
    NDOF = DOF*NN
    d = nodes[bars[:,1],:] - nodes[bars[:,0],:]
    L = np.sqrt((d**2).sum(axis=1))
    angle = d.T/L
    a = np.concatenate((-angle.T,angle.T),axis=1)
    K = np.zeros([NDOF,NDOF])
    for k in range(NE) :
        aux = 2*bars[k,:]
        index = np.r_[aux[0]:aux[0]+2,aux[1]:aux[1]+2]
        
        ES = np.dot(a[k][np.newaxis].T*E*S,a[k][np.newaxis]/L[k])
        K[np.ix_(index,index)] = K[np.ix_(index,index)] + ES
    
    freeDOF = DOFCON.flatten().nonzero()[0]
    supportDOF = (DOFCON.flatten() == 0).nonzero()[0]
    #print(supportDOF)
    
    Kff = K[np.ix_(freeDOF,freeDOF)]
    Kfr = K[np.ix_(freeDOF,supportDOF)]
    Krf = Kfr.T
    Krr = K[np.ix_(supportDOF,supportDOF)]
    Pf = P.flatten()[freeDOF]
    Uf = np.linalg.solve(Kff,Pf)
    U = DOFCON.astype(float).flatten()
    U[freeDOF] = Uf
    U[supportDOF] = Ur
    U = U.reshape(NN,DOF)
    u = np.concatenate((U[bars[:,0]],U[bars[:,1]]),axis=1)
    N = E*A/L[:]*(a[:]*u[:]).sum(axis=1)
    R = (Krf[:]*Uf).sum(axis=1) + (Krr[:]*Ur).sum(axis=1)
    R = R.reshape(2,DOF)
    return np.array(N),np.array(R),U

def truss_analyse3D():
    NN = len(nodes)
    NE = len(bars)
    DOF = 3
    NDOF = DOF*NN
    d = nodes[bars[:,1],:] - nodes[bars[:,0],:]
    L = np.sqrt((d**2).sum(axis=1))
    angle = d.T/L
    a = np.concatenate((-angle.T,angle.T),axis=1)
    K = np.zeros([NDOF,NDOF])
    for k in range(NE) :
        aux = 3*bars[k,:]
        index = np.r_[aux[0]:aux[0]+3,aux[1]:aux[1]+3]
        
        ES = np.dot(a[k][np.newaxis].T*E*S,a[k][np.newaxis]/L[k])
        K[np.ix_(index,index)] = K[np.ix_(index,index)] + ES
    
    freeDOF = DOFCON.flatten().nonzero()[0]
    supportDOF = (DOFCON.flatten() == 0).nonzero()[0]
    print(supportDOF)
    
    Kff = K[np.ix_(freeDOF,freeDOF)]
    Kfr = K[np.ix_(freeDOF,supportDOF)]
    Krf = Kfr.T
    Krr = K[np.ix_(supportDOF,supportDOF)]
    Pf = P.flatten()[freeDOF]
    Uf = np.linalg.solve(Kff,Pf)
    U = DOFCON.astype(float).flatten()
    U[freeDOF] = Uf
    U[supportDOF] = Ur
    U = U.reshape(NN,DOF)
    u = np.concatenate((U[bars[:,0]],U[bars[:,1]]),axis=1)
    N = E*A/L[:]*(a[:]*u[:]).sum(axis=1)
    R = (Krf[:]*Uf).sum(axis=1) + (Krr[:]*Ur).sum(axis=1)
    R = R.reshape(4,DOF)
    return np.array(N),np.array(R),U

if cas == 2 :
    N, R, U = truss_analyse2D()
    print("Axial Forces (positive = tension, negative = compression) :\n", N[np.newaxis].T)
    print("Reaction forces (positive = upward, negative = downward) : \n", R)
    print("Deformation at nodes \n", U)
    plot2D(nodes,'gray','--',1,'undeformed')
    scale = 1
    Dnodes = U*scale + nodes
    plot2D(Dnodes,'red','-',2,'deformed')
    #plt.show()
    #plt.savefig("Customized Surface Plot.png",dpi=300)
elif cas == 3 : 
    plot3D(nodes,'gray','--',1,'undeformed')
    N, R, U = truss_analyse3D()
    print("Axial Forces (positive = tension, negative = compression) :\n", N[np.newaxis].T)
    print("Reaction forces (positive = upward, negative = downward) : \n", R)
    print("Deformation at nodes \n", U)    
    scale = 1
    Dnodes = U*scale + nodes
    plot3D(Dnodes,'red','-',2,'deformed')
    
import msvcrt

while 0:
    print('Testing..')
    # body of the loop ...
    if msvcrt.kbhit():
        if ord(msvcrt.getch()) == 27:
            break
            
path = np.linspace(nodes,U*scale + nodes)

print(path)

fig, ax = plt.subplots()
line, = ax.plot([])   # A tuple unpacking to unpack the only plot
ax.set_xlim(0, 2*np.pi)
ax.set_ylim(-1.1, 1.1)
def animate(frame_num):
    y = np.sin(x + 2*np.pi * frame_num/100)
    line.set_data((x, y))
    return line
anim = FuncAnimation(fig, animate, frames=100, interval=20)


20


<IPython.core.display.Javascript object>

[ 3  4  5  6  7  8  9 10 11 12 13 14]
Axial Forces (positive = tension, negative = compression) :
 [[-0.07948303]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [-0.11945691]
 [-0.03997388]
 [-0.04335946]
 [-0.03997388]
 [-0.79483034]
 [ 1.6036023 ]
 [ 2.63542314]
 [ 2.36457686]
 [ 2.60228991]
 [ 2.39909878]
 [ 2.60228991]
 [ 2.39909878]
 [ 0.79879461]
 [-0.03402464]
 [ 0.03402464]
 [ 0.        ]]
Reaction forces (positive = upward, negative = downward) : 
 [[-7.94830345e-02  3.38557850e-03 -1.56974652e+00]
 [ 3.38557850e-03  0.00000000e+00 -2.66927893e+00]
 [ 0.00000000e+00  0.00000000e+00 -2.36457686e+00]
 [ 7.60974560e-02 -3.38557850e-03 -3.39639770e+00]]
Deformation at nodes 
 [[ 7.16063373e-05  0.00000000e+00  7.16063373e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.07618840e-04 -2.40485432e-01  1.44468676e-02]
 [ 2.

<IPython.core.display.Javascript object>

[[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 1.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 1.00000000e+00  1.00000000e+00  0.00000000e+00]
  ...
  [ 1.00000000e+00  1.00000000e+00  1.00000000e+01]
  [ 0.00000000e+00  1.00000000e+00  1.00000000e+01]
  [ 5.00000000e-01  5.00000000e-01  4.00000000e+01]]

 [[ 1.46135382e-06  0.00000000e+00  1.46135382e-04]
  [ 1.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 1.00000000e+00  1.00000000e+00  0.00000000e+00]
  ...
  [ 1.00428347e+00  9.95091399e-01  1.00004845e+01]
  [ 4.28426544e-03  9.99999265e-01  1.00004347e+01]
  [ 4.96971918e-01  4.88179260e-01  4.00016836e+01]]

 [[ 2.92270765e-06  0.00000000e+00  2.92270765e-04]
  [ 1.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 1.00000000e+00  1.00000000e+00  0.00000000e+00]
  ...
  [ 1.00856694e+00  9.90182798e-01  1.00009691e+01]
  [ 8.56853087e-03  9.99998530e-01  1.00008695e+01]
  [ 4.93943836e-01  4.76358521e-01  4.00033673e+01]]

 ...

 [[ 6.86836297e-05  0.00000000e+00

<IPython.core.display.Javascript object>